In [1]:
import os
from PIL import Image
import pandas as pd

In [2]:
df_path = "/Users/user/Documents/Coding/cro_location_intelligence/notebook/7-11 Location for Ford.xlsx"
df = pd.read_excel(df_path)
df.head()

,store_id,prov_namt,latitude,longitude
0,1025,กรุงเทพมหานคร,13.838724,100.575318
1,1026,ปทุมธานี,14.036545,100.733520
2,1028,ตรัง,7.567873,99.614094
3,1032,กรุงเทพมหานคร,13.719625,100.442061
4,1033,กรุงเทพมหานคร,13.768390,100.721288


In [3]:
import numpy as np

# add new column name embedding with nan
df["embedding"] = [[]] * len(df)
df.head()

,store_id,prov_namt,latitude,longitude,embedding
0,1025,กรุงเทพมหานคร,13.838724,100.575318,[]
1,1026,ปทุมธานี,14.036545,100.733520,[]
2,1028,ตรัง,7.567873,99.614094,[]
3,1032,กรุงเทพมหานคร,13.719625,100.442061,[]
4,1033,กรุงเทพมหานคร,13.768390,100.721288,[]


In [4]:
# check is store_id is unique
df["store_id"].is_unique

True

In [5]:
# do model inference

In [6]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print(
            "MPS not available because the current PyTorch install was not "
            "built with MPS enabled."
        )
    else:
        print(
            "MPS not available because the current MacOS version is not 12.3+ "
            "and/or you do not have an MPS-enabled device on this machine."
        )

else:
    device = torch.device("mps")
device

device(type='mps')

In [7]:
import timm
from geo_utils import (
    crop_middle_image_from_side_size,
    apply_circle_mask,
)

model = timm.create_model(
    "maxvit_tiny_tf_512.in1k",
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
).to(
    device
)  # Move the model to the GPU
model = model.eval()
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)


def preprocess_image(image: Image.Image) -> Image.Image:
    image = crop_middle_image_from_side_size(image, 2400, 1600)
    image = apply_circle_mask(image)
    return image


preprocessed_image_folder = (
    "/Users/user/Documents/Coding/cro_location_intelligence/notebook/data/crop_image"
)


def predict(model, transforms, image_path, preprocessed_image_path, device=device):
    image = Image.open(image_path).convert("RGB")
    preprocessed_image = preprocess_image(image)
    image_tensor = (
        transforms(preprocessed_image).unsqueeze(0).to(device)
    )  # Move input data to the GPU
    output = model(image_tensor)

    # or equivalently (without needing to set num_classes=0)
    output = model.forward_features(image_tensor)
    output = model.forward_head(output, pre_logits=True)
    embedding = output.detach().cpu().numpy()[0]
    preprocessed_image.save(preprocessed_image_path)
    # save preprocessed_image
    return embedding  # Move the result back to the CPU for further processing if needed

In [8]:
image_path = "/Users/user/Documents/Coding/cro_location_intelligence/notebook/data/full_image/1025.png"
image_path = "/Users/user/Documents/Coding/cro_location_intelligence/notebook/data/full_image/1026.png"
preprocessed_image_path = os.path.join(
    preprocessed_image_folder, os.path.basename(image_path)
)
result = predict(
    model,
    transforms,
    image_path,
    preprocessed_image_path,
)
result

array([-0.2934701 , -0.58131367,  0.36780483, -0.02461963, -0.7779438 ,
        0.7492058 ,  0.38792056,  0.47475913,  0.12855491,  0.76059544,
       -0.844794  , -0.08179618, -0.91854733, -0.2634485 , -0.7161958 ,
        0.8080405 ,  0.03396254,  0.58737653,  0.70723045, -0.9452553 ,
        0.595879  , -0.14017929, -0.6523275 , -0.44984165,  0.7311205 ,
       -0.37923956,  0.44274426,  0.10363052, -0.37624   , -0.68525517,
       -0.36235118, -0.10684091, -0.3427367 ,  0.4878487 ,  0.37937248,
       -0.6165281 , -0.7154109 ,  0.36468017, -0.04165799,  0.34470746,
        0.86408997,  0.24312724,  0.06242277, -0.8633072 , -0.7092936 ,
        0.4776551 , -0.35540318,  0.16013888,  0.23926508, -0.43989167,
       -0.9415436 , -0.2480825 , -0.39577952,  0.16052458,  0.32229835,
        0.18263163, -0.03658738, -0.18266593,  0.33793658,  0.54054534,
        0.38385743,  0.59827995, -0.48577666, -0.20710914, -0.2368067 ,
       -0.57510006,  0.8813019 ,  0.09484208,  0.6147746 , -0.56

In [9]:
index = 1
df.loc[index, "embedding"], pd.isna(df.loc[index, "embedding"])

([], array([], dtype=bool))

In [10]:
save_path = "./df_embedding.pkl"
import os
import pandas as pd

if os.path.exists(save_path):
    print("Load success df_embedding.pkl")
    df = pd.read_pickle(save_path)

In [11]:
df.head()

,store_id,prov_namt,latitude,longitude,embedding
0,1025,กรุงเทพมหานคร,13.838724,100.575318,[]
1,1026,ปทุมธานี,14.036545,100.733520,[]
2,1028,ตรัง,7.567873,99.614094,[]
3,1032,กรุงเทพมหานคร,13.719625,100.442061,[]
4,1033,กรุงเทพมหานคร,13.768390,100.721288,[]


In [12]:
# count embedding that is []
df["embedding"].apply(lambda x: x == []).sum()

11055

In [13]:
store_id_list = df["store_id"]
image_folder = (
    "/Users/user/Documents/Coding/cro_location_intelligence/notebook/data/full_image"
)
from tqdm import tqdm

image_paths = []
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    store_id = df.loc[index, "store_id"]
    # print(store_id)

    image_path = os.path.join(image_folder, f"{store_id}.png")
    if not os.path.exists(image_path):
        continue
    # is nan or not
    # print('df.loc[index, "embedding"]', df.loc[index, "embedding"])
    if df.loc[index, "embedding"] != []:
        continue
    preprocessed_image_path = os.path.join(
        preprocessed_image_folder, os.path.basename(image_path)
    )
    # if os.path.exists(preprocessed_image_path):
    #     continue
    image_paths.append(preprocessed_image_path)
batch_size = 4  # Choose an appropriate batch size

for i in tqdm(
    range(0, len(image_paths), batch_size), desc="Processing Images", leave=False
):
    batch_paths = image_paths[i : i + batch_size]

    batch_images = [Image.open(path).convert("RGB") for path in batch_paths]
    preprocessed_images = [preprocess_image(image) for image in batch_images]
    # image_tensors = torch.stack(
    #     [transforms(image).unsqueeze(0) for image in preprocessed_images]
    # ).to(device)
    image_tensors = (
        torch.stack([transforms(image).unsqueeze(0) for image in preprocessed_images])
        .squeeze(1)
        .to(device)
    )

    # Move input data to the GPU
    output = model(image_tensors)

    # or equivalently (without needing to set num_classes=0)
    output = model.forward_features(image_tensors)
    output = model.forward_head(output, pre_logits=True)
    embeddings = output.detach().cpu().numpy()

    for j, embedding in enumerate(embeddings):
        index = i + j  # Assuming 'index' is the index to update in the DataFrame
        df.embedding[index] = embedding.tolist()

Processing rows:   0%|          | 0/11055 [00:00<?, ?it/s]

Processing Images:   0%|          | 0/2500 [00:00<?, ?it/s]/var/folders/9g/3rr3k99j0td2974k71sy46_h0000gp/T/ipykernel_43177/1891159996.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.embedding[index] = embedding.tolist()


In [16]:
# save df to pickle
df.to_pickle(save_path)

In [17]:
# count embedding that is []
df["embedding"].apply(lambda x: x == []).sum()

1058